In [26]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [12]:
import json
import json_repair

from typing import List, Dict
from colorama import Fore

from haystack import component, Pipeline
from haystack.dataclasses import Document

from haystack.components.embedders import OpenAITextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.components.embedders import OpenAIDocumentEmbedder
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.writers import DocumentWriter

## Components
1. Initialize a component
2. Inspect expected input/output types

In [28]:
embedder = OpenAIDocumentEmbedder(model="text-embedding-3-small")

In [ ]:
embedder

In [ ]:
documents = [Document(content="Haystack is an open source AI framework to build full AI applications in Python"),
             Document(content="You can build AI Pipelines by combining Components"),]

embedder.run(documents=documents)

## Pipelines (Indexing + RAG Pipelines)

1. Add components
2. Connect components

In [ ]:
document_store = InMemoryDocumentStore()
document_store.count_documents()

In [32]:
fetcher = LinkContentFetcher()
converter = HTMLToDocument()
embedder = OpenAIDocumentEmbedder()
writer = DocumentWriter(document_store=document_store)

indexing_pipeline = Pipeline()

indexing_pipeline.add_component("fetcher", fetcher)
indexing_pipeline.add_component("converter", converter)
indexing_pipeline.add_component("embedder", embedder)
indexing_pipeline.add_component("writer", writer)

In [ ]:
indexing_pipeline.connect("fetcher", "converter")
indexing_pipeline.connect("converter", "embedder")
indexing_pipeline.connect("embedder", "writer")

In [ ]:
indexing_pipeline.show()

In [ ]:
indexing_pipeline.run({"fetcher": {"urls": ["https://haystack.deepset.ai/integrations/mistral", 
                                            "https://haystack.deepset.ai/integrations/cohere"]}})

In [ ]:
document_store.count_documents()

### RAG

In [45]:
prompt_template = """Answer the question given the following documents. 
Documents:
{% for document in documents %}
    {{document.content}}
{% endfor %}
Question: {{query}}
Answer:
 """

In [ ]:
query_embedder = OpenAITextEmbedder()
retriever = InMemoryEmbeddingRetriever(document_store=document_store)
prompt = PromptBuilder(template=prompt_template)
llm = OpenAIGenerator(model="gpt-4")

rag = Pipeline()

rag.add_component("query_embedder", query_embedder)
rag.add_component("retriever", retriever)
rag.add_component("prompt", prompt)
rag.add_component("llm", llm)


rag.connect("query_embedder.embedding", "retriever.query_embedding")
rag.connect("retriever.documents", "prompt.documents")
rag.connect("prompt", "llm")

In [ ]:
query = "What are the available Jina components?"
rag.run({"query_embedder": {"text": query},
         "prompt": {"query": query}})

## Self Reflection - Entity Extraction

In [49]:
template = """"Extract entities from the following text
Text: {{ text }} 
The entities should be presented as key value pairs in a JSON object.

{% if entites_to_validate %}
    Here are the entities you extracted: {{entites_to_validate[0]}}
    Have you extracted all possible entities? 
    If yes, say 'done' and return the extracted entites in the next line
    If not, say 'incomplete' and return the entities in the next line
{% endif %}
Entities:
"""

In [50]:
@component
class EntitiesValidator:
    
    @component.output_types(entites_to_validate=str, entities=str)
    def run(self, replies: List[str]):
        if 'done' in replies[0]:
            return {"entities":replies[0].replace('done', '')}
        else:
            print(Fore.RED + "Entities to Validate\n", replies[0])
            return {"entites_to_validate": replies[0]}

In [ ]:
prompt_template = PromptBuilder(template=template)
llm = OpenAIGenerator()
entities_validator = EntitiesValidator()

pipeline = Pipeline(max_loops_allowed=10)

pipeline.add_component("prompt_builder", prompt_template)
pipeline.add_component("entities_validator", entities_validator)
pipeline.add_component("llm", llm)

pipeline.connect("prompt_builder.prompt", "llm.prompt")
pipeline.connect("llm.replies", "entities_validator.replies")
pipeline.connect("entities_validator.entites_to_validate", "prompt_builder.entites_to_validate")

In [ ]:
pipeline.show()

In [25]:
text = """
June 19-20, 2024
Paris, France
Composing AI Applications as a Graph with Haystack - Tuana Celik, deepset
There are many moving parts in a fully functioning AI application, whether that be retrieval-augmented generation, agents, chatbots.. Haystack is an open source AI framework in Python, recently redesigned in an opinionated way, that takes the approach of building AI applications as a directed (cyclic) graph. In this talk, we will see how Haystack allows engineers to take full control over how they design their AI workflows, making use of pretty much all model providers and technologies out there.
Speakers
Tuana Celik
Developer Relations Lead, deepset
Tuana is a Developer Advocate at deepset where she focuses on the open source AI community and Haystack, deepset’s open source AI framework. She has been actively involved in helping and maintaining the Haystack developer community and Haystack’s educational materials.
"""

result = pipeline.run({"prompt_builder": {"text": text}})
print(Fore.GREEN + result['entities_validator']['entities'])

Entities to Validate
 {
  "Date": "June 19-20, 2024",
  "Location": "Paris, France",
  "Topic": "Composing AI Applications as a Graph with Haystack",
  "Speaker": "Tuana Celik",
  "Company": "deepset",
  "Role": "Developer Relations Lead",
  "Framework": "Haystack",
  "Language": "Python"
}
{
    "Date": "June 19-20, 2024",
    "Location": "Paris, France",
    "Topic": "Composing AI Applications as a Graph with Haystack",
    "Speaker": "Tuana Celik",
    "Company": "deepset",
    "Role": "Developer Relations Lead",
    "Framework": "Haystack",
    "Programming Language": "Python"
}




In [3]:
@component
class QuizParser:
    @component.output_types(quiz=Dict)
    def run(self, replies: List[str]):
        reply = replies[0]

        # even if prompted to respond with JSON only, sometimes the model returns a mix of JSON and text
        first_index = min(reply.find("{"), reply.find("["))
        last_index = max(reply.rfind("}"), reply.rfind("]")) + 1

        json_portion = reply[first_index:last_index]

        try:
            quiz = json.loads(json_portion)
        except json.JSONDecodeError:
            # if the JSON is not well-formed, try to repair it
            quiz = json_repair.loads(json_portion)

        # sometimes the JSON contains a list instead of a dictionary
        if isinstance(quiz, list):
            quiz = quiz[0]
        return {"quiz": quiz}

In [ ]:
quiz_generation_template = """Given the following text, create 5 multiple choice quizzes in JSON format.
Each question should have 4 different options, and only one of them should be correct.
The options should be unambiguous.
Each option should begin with a letter followed by a period and a space (e.g., "a. option").
The question should also briefly mention the general topic of the text so that it can be understood in isolation.
Each question should not give hints to answer the other questions.
Include challenging questions, which require reasoning.

respond with JSON only, no markdown or descriptions.

example JSON format you should absolutely follow:
{"topic": "a sentence explaining the topic of the text",
 "questions":
  [
    {
      "question": "text of the question",
      "options": ["a. 1st option", "b. 2nd option", "c. 3rd option", "d. 4th option"],
      "right_option": "c"  # letter of the right option ("a" for the first, "b" for the second, etc.)
    }, ...
  ]
}

text:
{% for doc in documents %}{{ doc.content|truncate(4000) }}{% endfor %}
"""


quiz_generation_pipeline = Pipeline()
quiz_generation_pipeline.add_component("link_content_fetcher", LinkContentFetcher())
quiz_generation_pipeline.add_component("html_converter", HTMLToDocument())
quiz_generation_pipeline.add_component(
    "prompt_builder", PromptBuilder(template=quiz_generation_template)
)
quiz_generation_pipeline.add_component(
    "generator",
    OpenAIGenerator(model="gpt-4"),
)
quiz_generation_pipeline.add_component("quiz_parser", QuizParser())

quiz_generation_pipeline.connect("link_content_fetcher", "html_converter")
quiz_generation_pipeline.connect("html_converter", "prompt_builder")
quiz_generation_pipeline.connect("prompt_builder", "generator")
quiz_generation_pipeline.connect("generator", "quiz_parser")

In [ ]:
quiz_generation_pipeline.show()

In [11]:
url = "https://www.rainforest-alliance.org/species/capybara/"
quiz = quiz_generation_pipeline.run({"link_content_fetcher": {"urls": [url]}})

print(json.dumps(quiz['quiz_parser']['quiz'], indent=2))

{
  "topic": "Information about Capybaras, including their anatomy, habitat, diet, threats, and interesting facts",
  "questions": [
    {
      "question": "What kind of extremes in size can Capybaras grow up to?",
      "options": [
        "a. They can weigh more than 200 lbs and grow to be 6 feet long and 3 feet tall",
        "b. They can weigh more than 100 lbs and grow to be 4 feet long and 2 feet tall",
        "c. They can weigh more than 50 lbs and grow to be 2 feet long and 1 feet tall",
        "d. They can weigh more than 150 lbs and grow to be 5 feet long and 2.5 feet tall"
      ],
      "right_option": "b"
    },
    {
      "question": "What is the threat posed by humans to Capybaras?",
      "options": [
        "a. Humans pollute their habitats",
        "b. Humans use them for entertainment purposes",
        "c. Humans hunt them for their meat and hide",
        "d. Humans displace them from their natural habitats"
      ],
      "right_option": "c"
    },
    {
  